In [35]:
import requests
import pandas as pd

# acquire data

In [3]:
# note that this had to strip the last two parameters from the api url in the google doc
# also upped the limit 
api_url = "https://opendata.maryland.gov/api/id/crti-ybyp.json?$select=*&$order=`:id`+ASC&$limit=3000&$offset=0 "
response = requests.get(api_url)
data = response.json()

In [4]:
data[0:2]

[{'sno': '1',
  'complaint': '232216',
  'complaint_description': 'Odor Complaint',
  'complaint_type': 'Odor',
  'recieved_date': '2024-03-01',
  'incident_date': '2024-03-01',
  'county': 'Dorchester',
  'incident_closed_date': '2024-03-01',
  'incident_status_desc': 'Incident Closed - No further action',
  'incident_zip': '21835'},
 {'sno': '2',
  'complaint': '232215',
  'complaint_description': 'Odor Complaint',
  'complaint_type': 'Odor',
  'recieved_date': '2024-03-01',
  'incident_date': '2024-03-01',
  'county': 'Dorchester',
  'incident_closed_date': '2024-03-01',
  'incident_status_desc': 'Incident Closed - No further action',
  'incident_zip': '21835'}]

In [5]:
df = pd.DataFrame(data)

In [6]:
df

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,county,incident_closed_date,incident_status_desc,incident_zip
0,1,232216,Odor Complaint,Odor,2024-03-01,2024-03-01,Dorchester,2024-03-01,Incident Closed - No further action,21835
1,2,232215,Odor Complaint,Odor,2024-03-01,2024-03-01,Dorchester,2024-03-01,Incident Closed - No further action,21835
2,3,232214,Odor Complaint,Odor,2024-02-26,2024-02-26,Dorchester,2024-02-26,Incident Closed - No further action,21835
3,4,232205,Neighbor's fireplace is producing large amount...,"Air, Smoke",2024-02-29,2024-02-29,Carroll,NaN,Referred to Outside Agency,NaN
4,5,232204,Concern of loud boom noise in the Cheverly area,Other,2024-02-28,2024-02-28,Prince George's,2024-03-01,Incident Closed-Managed,NaN
...,...,...,...,...,...,...,...,...,...,...
1502,1503,8058,Boat company sandblasting boats and blowing wh...,Fugitive Dust/Particulate Matter,2021-01-11,2021-01-11,Queen Anne's,2022-03-08,Incident Closed - No further action,NaN
1503,1504,8057,Smoke and ash from neighbor's open burning in ...,Air,2021-01-07,2021-01-07,Anne Arundel,2021-01-07,Incident Closed-No Violation Observed,20751
1504,1505,8056,Smoke from neighbor's wood burning stove.,Air,2021-01-07,2021-01-04,Charles,NaN,Under Investigation,NaN
1505,1506,8051,Concern of materials being burned in fireplace...,Air,2021-01-04,2020-12-28,Frederick,2021-01-15,Incident Closed-No Violation Observed,21770


# clean data

In [40]:
# Before we do anything else, let's create a backup dataframe that we'll never manipulate.
backup_df = df.copy()

In [41]:
# Let's look at the data types for each column.
# df.dtypes

# That's a lot of "Objects" (meaning strings)! Let's convert the 'received date' to actual dates.
df['recieved_date'] = pd.to_datetime(df['recieved_date'])

# While we're at it, how about we do the same for the two other dates?
df['incident_date'] = pd.to_datetime(df['incident_date'])
df['incident_closed_date'] = pd.to_datetime(df['incident_closed_date'])

In [42]:
# # There are some `NaN` values in the county and ZIP code columns.
# # Let's toss them both out from the main dataframe. First, we'll see how many rows are missing one value or the other.
# df["county"].isna().sum()
# df["incident_zip"].isna().sum()


# # OK, so maybe we don't want to get rid of null ZIP codes. This is why we check!

# # We _can_ remove invalid county names, though.
# # We know we have just 12 NAs, but what about others? Let's look at all unique county values first.
# df["county"].unique()


# It looks like there are three more we should screen out: "Not Yet Determined", "Outside of Maryland" and "Statewide."
# do a new dataframe with only the values we expect.
df = df[~df['county'].isnull()]
df = df[~df['county'].isin(["Not Yet Determined", "Outside of Maryland", "Statewide"])]

In [43]:
# # We'll create other dataframes with just those values — and export them to CSV files so that we have the option of looking through those one by one at some point.
# null_county_rows = backup_df[backup_df['county'].isnull()]
# wrong_county_rows = backup_df[backup_df['county'].isin(["Not Yet Determined", "Outside of Maryland", "Statewide"])]

# Add them together and then export.
all_excluded_rows = pd.concat([null_county_rows, wrong_county_rows])

all_excluded_rows.to_csv("exported_data/excluded_counties.csv")

# analyze data

In [56]:
# For the summaries we're running, we'll limit that to just complaints received in the last year?
today = pd.to_datetime("now")
one_year_ago = today - pd.Timedelta(366, "day")

last_year_df = df[df["recieved_date"].dt.day > one_year_ago.day].copy()

# last_year_df["county"].value_counts()

In [57]:
# What are the most common types of complaint?
# last_year_df["complaint_type"]
last_year_df["complaint_type"].value_counts()

complaint_type
Odor                                     77
Air                                      56
Other                                    39
Fugitive Dust/Particulate Matter         19
Fumes                                     8
Smoke                                     8
Open Burning                              6
Air, Fumes, Odor                          6
Air, Odor                                 3
Air, Fugitive Dust/Particulate Matter     2
Odor, Smoke                               2
Fumes, Odor                               1
Waste                                     1
Fumes, Open Burning                       1
Air, Noise Complaint                      1
ARA AQCP non-regulated entity             1
Suspected Operation without Permit        1
Air, Open Burning                         1
Open Burning, Smoke                       1
Air, Fumes                                1
Air, Fumes, Smoke                         1
Air, Fumes, Odor, Smoke                   1
Fumes, Open Burni

In [59]:
# Wait, though. The second line has a type of "Air, Other" — shouldn't that count toward both categories?
# The data's bunched up, so we need to make the table longer.
# (Important caveat to be thinking what each row represents — you could fall into a trap with this transformation.)
last_year_df["complaint_type"] = last_year_df["complaint_type"].str.split(', ')
complaints_by_type = last_year_df.explode("complaint_type")

In [69]:
# Now we can see how many times each type of complaint was lodged.
last_year_complaint_frequency = complaints_by_type["complaint_type"].value_counts()

last_year_complaint_frequency

# We'll leave the complaints-by-type dataset there for now, but note that we _could_ export one CSV per complaint type,
# if for example that would help reporters on different beats focus on their relevant parts of the same data.

complaint_type
Odor                                  91
Air                                   74
Other                                 40
Fugitive Dust/Particulate Matter      21
Fumes                                 21
Smoke                                 15
Open Burning                          10
Air Pollutant Release                  1
Suspected Operation without Permit     1
ARA AQCP non-regulated entity          1
Waste                                  1
Noise Complaint                        1
Asbestos Complaint                     1
Name: count, dtype: int64

In [67]:
# Let's go back to the "one row = one complaint" dataset, and look at how many complaints have come in from each county.
last_year_by_county = last_year_df["county"].value_counts()

last_year_by_county

# export data

In [80]:
# Let's write out our main dataset as a CSV.
df.to_csv(
    f"exported_data/complaints.csv",
    index=False,
    columns=[
        "complaint",
        "incident_date",
        "county",
        "incident_zip",
        "complaint_type",
        "complaint_description",
        "recieved_date",
        "incident_status_desc",
        "incident_closed_date",
    ]
)


In [75]:
# We've also effectively made a couple of pivot tables (reports per county and frequency of report type). Let's export them, too.
last_year_complaint_frequency.to_csv("exported_data/complaint_frequency.csv")
last_year_by_county.to_csv("exported_data/county_frequency.csv")